In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [3]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [4]:
class WideAndDeepModel(keras.models.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

model = WideAndDeepModel(30, activation = "relu")

In [5]:
model.compile(loss = "mse", 
              loss_weights = [0.9, 0.1], 
              optimizer = keras.optimizers.SGD(learning_rate=1e-3))

history = model.fit((X_train_A, X_train_B), 
                    (y_train, y_train), 
                    epochs = 10,
                    validation_data = ((X_valid_A, X_valid_B), (y_valid, y_valid)))

total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))

y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/10
363/363 [==============================] - 3s 3ms/step - loss: 2.3244 - output_1_loss: 2.0837 - output_2_loss: 4.4902 - val_loss: 1.7365 - val_output_1_loss: 1.5372 - val_output_2_loss: 3.5302
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.9081 - output_1_loss: 0.7096 - output_2_loss: 2.6954 - val_loss: 0.9055 - val_output_1_loss: 0.6830 - val_output_2_loss: 2.9080
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.7607 - output_1_loss: 0.6241 - output_2_loss: 1.9907 - val_loss: 0.7794 - val_output_1_loss: 0.5763 - val_output_2_loss: 2.6080
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6942 - output_1_loss: 0.5863 - output_2_loss: 1.6656 - val_loss: 0.7318 - val_output_1_loss: 0.5485 - val_output_2_loss: 2.3815
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.6528 - output_1_loss: 0.5592 - output_2_loss: 1.4960 - val_loss: 0.6985 - val_output_1_loss: 0.5326 - val_ou